## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [211]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [212]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [213]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

## Download

Retrieve the data for both Yuba and Sutter counties

In [214]:
url = "https://services5.arcgis.com/THtdW72WxYCCmIVL/ArcGIS/rest/services/Cases_Weekly_Public/FeatureServer/1/query?where=1%3D1&objectIds=&time=&resultType=none&outFields=*&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&sqlFormat=none&f=pjson&token="

In [215]:
r = requests.get(url)

In [216]:
data = r.json()

## Parse

In [217]:
dict_list = []

In [218]:
for item in data["features"]:
    d = dict(item["attributes"])
    dict_list.append(d)

In [219]:
df = pd.DataFrame(dict_list)

In [220]:
df = df[
    [
        "reportdt",
        "name",
        "LiveOak",
        "YubaCIty",
        "SutterOther",
        "Marysville",
        "OlivehurstLinda",
        "Wheatland",
        "YubaOther",
        "PlumasLake",
    ]
]

Set the date

In [221]:
df["reportdt"] = df["reportdt"] / 1000

In [222]:
df["reportdt"] = pd.to_datetime(df["reportdt"], unit="s")

In [223]:
df["county_date"] = df["reportdt"].dt.date

Get the latest date in the timeseries

In [224]:
latest_df = df.loc[df.groupby(["name", "county_date"]).reportdt.idxmax()]

Sift out Yuba County's cities

In [225]:
yuba_df = latest_df[latest_df.name.eq("Yuba County")]

In [226]:
yuba_df = yuba_df[
    [
        "Marysville",
        "OlivehurstLinda",
        "PlumasLake",
        "Wheatland",
        "YubaOther",
        "county_date",
        "reportdt",
    ]
]

In [227]:
yuba_latest = yuba_df[yuba_df.reportdt == yuba_df.reportdt.max()]

Melt down

In [228]:
yuba_latest.set_index("county_date", inplace=True)

In [229]:
yuba_melt = pd.melt(
    yuba_latest.reset_index(),
    id_vars="county_date",
    var_name="area",
    value_name="confirmed_cases",
)

Clean names

In [230]:
yuba_clean = yuba_melt[yuba_melt["area"] != "reportdt"]

In [231]:
clean_yuba_cities = {
    "OlivehurstLinda": "Olivehurst Linda",
    "PlumasLake": "Plumas Lake",
    "YubaOther": "Other",
}

In [232]:
yuba_clean = yuba_clean.replace({"area": clean_yuba_cities})

In [233]:
yuba_clean.insert(0, "county", "Yuba")
yuba_clean

,county,county_date,area,confirmed_cases
0,Yuba,2021-09-11,Marysville,4356.0
1,Yuba,2021-09-11,Olivehurst Linda,2563.0
2,Yuba,2021-09-11,Plumas Lake,1099.0
3,Yuba,2021-09-11,Wheatland,500.0
4,Yuba,2021-09-11,Other,733.0


## Vet

In [234]:
try:
    assert not len(yuba_clean) > 5
except AssertionError:
    raise AssertionError("Yuba County's scraper has extra rows")

In [235]:
try:
    assert not len(yuba_clean) < 5
except AssertionError:
    raise AssertionError("Yuba County's scraper is missing rows")

## Export

Set date

In [236]:
tz = pytz.timezone("America/Los_Angeles")

In [237]:
today = datetime.now(tz).date()

In [238]:
slug = "yuba"

In [239]:
yuba_clean.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [240]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [241]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

ValueError: Missing column provided to 'parse_dates': 'county_date'

In [ ]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [ ]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)